# YouTube Audio Crawler - Colab v3 (FIXED - Cookie Auth)

**BREAKTHROUGH:** Cookie authentication bypasses YouTube bot detection!

**Requirements:**
- Format: M4A (AAC codec)
- Sample Rate: 44000 Hz (exact)
- Channels: Mono (1 channel)
- Bitrate: 192kbps

## ⚠️ IMPORTANT: Cookie Authentication Required

### How to get YouTube cookies:

**Chrome Extension:**
1. Install: [Get cookies.txt LOCALLY](https://chrome.google.com/webstore/detail/get-cookiestxt-locally/cclelndahbckbenkjhflpdbgdldlbecc)
2. Login to YouTube
3. Export cookies
4. Upload in Step 2

## Step 1: Install Dependencies

In [ ]:
!pip install -q yt-dlp
!ffmpeg -version | head -n 1
print("\n✅ Dependencies installed")

## Step 2: Upload YouTube Cookies

In [ ]:
from google.colab import files

print("📤 Upload your youtube_cookies.txt file:\n")
uploaded = files.upload()

COOKIES_FILE = None
for filename in uploaded.keys():
    if 'cookie' in filename.lower() or filename.endswith('.txt'):
        COOKIES_FILE = filename
        with open(filename, 'r') as f:
            lines = f.readlines()
            if any('youtube.com' in line for line in lines):
                print(f"\n✅ Valid YouTube cookies: {filename} ({len(lines)} lines)")
            else:
                print("⚠️  Warning: No youtube.com cookies found")
        break

if not COOKIES_FILE:
    print("\n❌ No cookies uploaded! Downloads may fail.")

## Step 3: Setup

In [ ]:
import yt_dlp
import json
import time
import random
from pathlib import Path
from datetime import datetime

DOWNLOADS_DIR = Path('./downloads')
DOWNLOADS_DIR.mkdir(exist_ok=True)

# Conservative sleep settings (cookies reduce need for aggressive throttling)
SLEEP_BETWEEN_VIDEOS = 10  # seconds between videos
SLEEP_MIN = 5   # minimum random sleep
SLEEP_MAX = 10  # maximum random sleep

print("✅ Setup complete")
print(f"📁 Downloads: {DOWNLOADS_DIR.absolute()}")
print(f"🍪 Cookies: {COOKIES_FILE if COOKIES_FILE else 'None'}")

## Step 4: Download Functions

In [ ]:
def get_channel_videos(channel_url):
    """Get all video URLs from channel"""
    print(f"\n🔍 Fetching: {channel_url}")
    
    ydl_opts = {
        'quiet': True,
        'extract_flat': True,
    }
    
    if COOKIES_FILE:
        ydl_opts['cookiefile'] = COOKIES_FILE
        print(f"   🍪 Using cookies")
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            result = ydl.extract_info(channel_url, download=False)
            
            if 'entries' in result:
                videos = []
                for entry in result['entries']:
                    if entry:
                        videos.append({
                            'video_id': entry.get('id'),
                            'url': f"https://www.youtube.com/watch?v={entry.get('id')}",
                            'title': entry.get('title', 'Unknown')
                        })
                
                print(f"   ✅ Found {len(videos)} videos")
                return videos
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
    
    return []

def download_video_audio(video_info):
    """Download audio with cookie auth"""
    video_id = video_info['video_id']
    video_url = video_info['url']
    title = video_info['title']
    
    print(f"\n⬇️  {title[:60]}...")
    print(f"   ID: {video_id}")
    
    video_dir = DOWNLOADS_DIR / video_id
    video_dir.mkdir(exist_ok=True)
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': str(video_dir / f"{video_id}.%(ext)s"),
        'no_warnings': False,  # Show warnings for debugging
        'ignoreerrors': False,
        # NO RATE LIMIT - cookies handle throttling naturally
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'm4a',
            'preferredquality': '192',
        }],
        'postprocessor_args': [
            '-ar', '44000',  # 44kHz exact
            '-ac', '1',      # Mono
        ],
    }
    
    if COOKIES_FILE:
        ydl_opts['cookiefile'] = COOKIES_FILE
        print(f"   🍪 Authenticated")
    
    try:
        # Small random delay
        sleep_time = random.uniform(SLEEP_MIN, SLEEP_MAX)
        print(f"   ⏳ Sleeping {sleep_time:.1f}s...")
        time.sleep(sleep_time)
        
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            
            # Save metadata
            metadata = {
                'video_id': video_id,
                'title': info.get('title'),
                'description': info.get('description'),
                'uploader': info.get('uploader'),
                'upload_date': info.get('upload_date'),
                'duration': info.get('duration'),
                'view_count': info.get('view_count'),
                'like_count': info.get('like_count'),
                'channel_id': info.get('channel_id'),
                'channel_url': info.get('channel_url'),
                'url': video_url,
                'audio_format': 'm4a',
                'sample_rate': 44000,
                'channels': 1,
                'bitrate': 192,
                'downloaded_at': datetime.now().isoformat(),
                'authentication': 'cookies' if COOKIES_FILE else 'none'
            }
            
            with open(video_dir / f"{video_id}.json", 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)
            
            print(f"   ✅ Success!")
            time.sleep(SLEEP_BETWEEN_VIDEOS)
            return True, None
            
    except Exception as e:
        error_msg = str(e)
        print(f"   ❌ Error: {error_msg[:100]}")
        return False, error_msg

print("✅ Functions loaded")

## Step 5: Configure Channels

In [ ]:
CHANNELS = [
    "https://www.youtube.com/channel/UCLFgJS-f6UKOJ3Xz0K8Kosg",  # leon
    # "https://www.youtube.com/channel/UC9nijyKbu2cQ0lrK6RyGLsw",  # 詩詩fly
]

MAX_VIDEOS_PER_CHANNEL = 3  # Test with 3 videos first

print(f"📋 {len(CHANNELS)} channels")
print(f"📊 Max {MAX_VIDEOS_PER_CHANNEL} videos per channel")

## Step 6: Start Download

In [ ]:
stats = {
    'total': 0,
    'success': 0,
    'failed': 0,
    'errors': {'403': 0, 'bot': 0, 'other': 0},
    'start': datetime.now()
}

print("🚀 Starting...")
print(f"⏰ {stats['start'].strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🍪 Cookies: {'✅ Yes' if COOKIES_FILE else '❌ No'}")
print("="*60)

for channel_url in CHANNELS:
    print(f"\n📺 Channel: {channel_url}")
    
    videos = get_channel_videos(channel_url)
    if not videos:
        continue
    
    if MAX_VIDEOS_PER_CHANNEL:
        videos = videos[:MAX_VIDEOS_PER_CHANNEL]
    
    for i, video in enumerate(videos, 1):
        print(f"\n--- Video {i}/{len(videos)} ---")
        stats['total'] += 1
        
        success, error = download_video_audio(video)
        
        if success:
            stats['success'] += 1
        else:
            stats['failed'] += 1
            if error:
                if '403' in error or 'Forbidden' in error:
                    stats['errors']['403'] += 1
                elif 'bot' in error.lower():
                    stats['errors']['bot'] += 1
                else:
                    stats['errors']['other'] += 1

stats['end'] = datetime.now()
stats['duration'] = (stats['end'] - stats['start']).total_seconds()

print("\n" + "="*60)
print("📊 RESULTS")
print("="*60)
print(f"⏰ Duration: {stats['duration']:.0f}s ({stats['duration']/60:.1f}min)")
print(f"📹 Total: {stats['total']}")
print(f"✅ Success: {stats['success']}")
print(f"❌ Failed: {stats['failed']}")
print(f"\nErrors:")
print(f"  403 Forbidden: {stats['errors']['403']}")
print(f"  Bot detection: {stats['errors']['bot']}")
print(f"  Other: {stats['errors']['other']}")

if stats['total'] > 0:
    rate = (stats['success'] / stats['total']) * 100
    print(f"\n📈 Success Rate: {rate:.1f}%")
    
    if rate > 0:
        print("\n🎉 BREAKTHROUGH! Cookie authentication works!")
        print("   No need for paid IP rotator!")
    else:
        print("\n⚠️  Troubleshooting needed - check error types above")

## Step 7: View Downloaded Files

In [ ]:
print("📂 Downloaded files:\n")

success_count = 0
total_mb = 0

for vdir in sorted(DOWNLOADS_DIR.iterdir()):
    if not vdir.is_dir():
        continue
    
    m4a = list(vdir.glob('*.m4a'))
    
    if m4a:
        success_count += 1
        size = m4a[0].stat().st_size / (1024*1024)
        total_mb += size
        print(f"✅ {vdir.name}")
        print(f"   └─ {m4a[0].name} ({size:.2f} MB)")
    else:
        print(f"❌ {vdir.name} (incomplete)")

print(f"\n📊 Summary: {success_count} files, {total_mb:.2f} MB total")
if success_count > 0:
    print(f"📏 Average: {total_mb/success_count:.2f} MB per file")

## Step 8: Create ZIP (Optional)

In [ ]:
import shutil

zip_name = f"youtube_audio_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
shutil.make_archive(zip_name, 'zip', DOWNLOADS_DIR)
print(f"📦 Created: {zip_name}.zip")

try:
    files.download(f"{zip_name}.zip")
    print("⬇️  Download started!")
except:
    print("💡 Download from files panel (left sidebar)")